# 前提条件


* 学習用の設定ファイル及び教師データ、学習データが作成済みであること  
※私も以下の記事を参考にさせていただきました。  
[YOLOを使ってドアラ検出〜後編〜](https://melheaven.hatenadiary.jp/entry/yolo-darknet2)
* ハイパーパラメータは公式の説明を参照ください  
[How to train (to detect your custom objects)](https://github.com/AlexeyAB/darknet#how-to-train-with-multi-gpu)

# google driveにマウント

In [ ]:
# セルを実行後、表示されるURL先に表示されるパスワードを入力すると Google ドライブをマウントし、ドライブ内のファイルを使用できるようになります。
from google.colab import drive
drive.mount('/content/drive')

# 作業フォルダに移動

In [ ]:
# フォルダは事前に作成する
%cd '/content/drive/MyDrive/hogehoge'
%pwd && ls

# darknet環境構築

In [ ]:
!git clone https://github.com/AlexeyAB/darknet/
% cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt update
!apt-get install libopencv-dev
!apt-get install libcanberra-gtk-module

In [ ]:
# 確認
%ls

# コンパイル

In [ ]:
# 2回目以降は!make clean && make
!make

# 学習済みモデル（重み）取得
* 以下ではyolov4-tiny.weightを使用します  
  モデルに応じて必要なモデルを公式からDLしてください。
* 参考  
wget https://pjreddie.com/media/files/yolov3-tiny.weights  
wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [ ]:
%mkdir weight
%cd weight
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [ ]:
%ls

# サンプル画像で物体検出

In [ ]:
# -dont_showをつけて下記コマンドを実行する。redictions.jpgができていれば成功。
# -dont_showをつけない場合、下記が表示される。これはGoogleCorab環境でOpenCVで画像が表示できないめだと思います。
# Unable to init server: Could not connect: Connection refused
# (predictions:2467): Gtk-WARNING **: 02:13:15.482: cannot open display: 
# https://github.com/pjreddie/darknet/issues/1600
%cd ../
!./darknet detector test cfg/coco.data cfg/yolov4-tiny.cfg weight/yolov4-tiny.weights data/dog.jpg -dont_show >> test_dog.txt
# logに出力しなければPredicted結果も表示されます。

# 学習
* 以下の記事を参考にさせていただきました。  
[Google Colaboratory で物体検出モデルを作成（後編）](https://lab.m-field.co.jp/2020/03/17/google-colaboratory-back-part/)

In [ ]:
# 事前学習済みweightsファイルのダウンロードし、google driveの実行環境にアップロードする
# https://github.com/AlexeyAB/darknet#how-to-train-with-multi-gpu
# How to train (to detect your custom objects) > yolov4.conv.137
# コマンド形式： darknet detector train <data> <cfg> <weights>
# 例
!./darknet detector train cfg/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show >> custom_train.txt

In [ ]:
# 学習の中断とloss値の確認
# セル停止ボタンを押下する
import matplotlib.pyplot as plt
%matplotlib inline
def plot_yolo_log(logfile, xlim=None, x_scale=None, y_scale=None):
    """
    ログファイルからlossの値をプロットする関数
    logfile: ログファイルのパス
    xlim: x軸の表示範囲をタプルで指定
    x_scale: x軸のスケール
    y_scale: y軸のスケール
    """
    with open(logfile, 'r') as f:
        lines  = [line.rstrip("\n") for line in f]
    numbers = {'1','2','3','4','5','6','7','8','9'}
    iters = []
    loss = []
    fig,ax = plt.subplots()
    prev_line = ""
    for line in lines:
        args = line.split(' ')
        if len(args) < 4:
            continue
        if args[1][-1:]==':' and args[1][0] in numbers :
            #print(args)
            iters.append(int(args[1][:-1]))            
            loss.append(float(args[3]))
    
    if xlim:
        ax.set_xlim(xlim)
    if x_scale:
        ax.set_xscale(x_scale)
    if y_scale:
        ax.set_yscale(y_scale)
    ax.plot(iters,loss)
    plt.xlabel('iters')
    plt.ylabel('loss')
    plt.grid()
    ticks = range(0,250,10)
    #ax.set_yticks(ticks)
    plt.show()
plot_yolo_log("train_log.txt", xlim=(0, 1000), y_scale='log',)

In [ ]:
# 学習の再開
# 保存されたweights ファイルを用いて学習を再開
# 例
!./darknet detector train cfg/obj.data cfg/yolov4-custom.cfg data/airship/backup/yolov4-custom_last.weights -dont_show >> custom_train.txt

In [ ]:
# 認識精度確認
# 実行形式 darknet detector map <.data> <.cfg> <weight> 
# 例
!./darknet detector map cfg/obj.data cfg/yolov4-custom.cfg data/airship/backup/yolov4-custom_last.weights